# Movie Data Cleaning from SQLite Database

In [31]:
import pandas as pd
import numpy as np
import sqlite3 as ss

## 1. Load Data from SQLite Database

Connects to the SQLite database file `movies_data/bollywood_movies.sqlite` and reads the entire `Movies` table into a pandas DataFrame.

In [32]:
try:
    # Establish connection to the SQLite database
    db_path = "/home/nineleaps/Documents/Python_Training/Final_test/movies_data/raw_data/bollywood_movies.sqlite"
    con = ss.connect(db_path)
    
    # Define the SQL query to select all data from the 'Movies' table
    query = "SELECT * from Movies "
    
    # Read data using pandas
    df = pd.read_sql_query(query, con)
    
    print(f"Successfully loaded data from {db_path}, table 'Movies'.")
    print(f"DataFrame shape: {df.shape}")

except ss.Error as e:
    print(f"SQLite error: {e}")
    # You might want to stop execution or handle the error appropriately
    # For now, we'll create an empty DataFrame to avoid downstream errors
    df = pd.DataFrame() 
    print("Created an empty DataFrame due to DB connection/query error.")
    
except FileNotFoundError:
    print(f"Error: Database file not found at {db_path}")
    print("Please ensure 'bollywood_movies.sqlite' exists in the 'movies_data' subfolder.")
    df = pd.DataFrame()
    print("Created an empty DataFrame.")

finally:
    # Ensure the connection is closed even if errors occur
    if 'con' in locals() and con:
        con.close()
        print("Database connection closed.")

Successfully loaded data from /home/nineleaps/Documents/Python_Training/Final_test/movies_data/raw_data/bollywood_movies.sqlite, table 'Movies'.
DataFrame shape: (55, 14)
Database connection closed.


## 2. Inspect Initial State

Let's look at the first few rows, data types, and missing values of the loaded data.

In [33]:
# Only proceed if the DataFrame was loaded successfully
if not df.empty:
    print("--- Initial Data ---")
    # Display the first 5 rows
    print("\nFirst 5 rows (Initial):")
    display(df.head())
    
    # Print initial info
    print("\nInitial DataFrame Info:")
    df.info()
    
    # Print initial missing values
    print("\nInitial Missing values per column:")
    # Calculate and print missing values only if there are columns
    if not df.columns.empty:
        print(df.isna().sum().sort_values(ascending=False))
    else:
        print("DataFrame has no columns to check for missing values.")
    print("-" * 30) # Separator
else:
    print("DataFrame is empty. Cannot display initial state. Please check the data loading step.")

--- Initial Data ---

First 5 rows (Initial):


,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
0,0,MOV001,Lagaan,Ashutosh Gowariker,None,2001,NaN,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions
1,1,MOV002,Kabhi Khushi Kabhie Gham...,Karan Johar,Family Drama,2001,50.0,100.0,7.9,210,Shah Rukh Khan,Kajol,Hindi,Dharma Productions
2,2,MOV003,M.S. Dhoni: The Untold Story,Neeraj Pandey,Biographical Sports Drama,2016,NaN,215.0,7.9,184,Sushant Singh Rajput,Kiara Advani,Hindi,Fox Star Studios
3,3,MOV004,Baahubali 2: The Conclusion,S. S. Rajamouli,Epic Fantasy Action,2017,250.0,1810.0,8.2,171,Prabhas,Anushka Shetty,Telugu,Arka Media Works
4,4,MOV005,Chennai Express,Rohit Shetty,Action Comedy,2013,NaN,423.0,6.4,141,Shah Rukh Khan,Deepika Padukone,Hindi,Red Chillies Entertainment



Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               55 non-null     int64  
 1   MovieID             55 non-null     object 
 2   Title               55 non-null     object 
 3   Director            55 non-null     object 
 4   Genre               54 non-null     object 
 5   ReleaseYear         55 non-null     int64  
 6   Budget (Crores)     43 non-null     float64
 7   BoxOffice (Crores)  46 non-null     float64
 8   Rating              55 non-null     float64
 9   Duration (minutes)  55 non-null     int64  
 10  LeadActor           55 non-null     object 
 11  LeadActress         55 non-null     object 
 12  Language            55 non-null     object 
 13  ProductionCompany   55 non-null     object 
dtypes: float64(3), int64(3), object(8)
memory usage: 6.1+ KB

Initial Missing values pe

## 3. Data Cleaning Steps

Apply the cleaning procedures.

In [34]:
# Only proceed if the DataFrame is not empty
if not df.empty:
    print("\n--- Starting Data Cleaning ---")
    
    # Step 1: Drop the redundant 'index' column (if it exists from the DB import)
    print("\nStep 1: Dropping redundant 'index' column (if exists)...")
    if 'index' in df.columns:
        df.drop('index', axis=1, inplace=True)
        print("Column 'index' dropped.")
    else:
        print("Column 'index' not found, skipping drop.")
else:
     print("DataFrame is empty. Skipping cleaning steps.")


--- Starting Data Cleaning ---

Step 1: Dropping redundant 'index' column (if exists)...
Column 'index' dropped.


In [35]:
# Step 2: Rename columns to snake_case (adjust names based on actual columns from DB)
if not df.empty:
    print("\nStep 2: Renaming columns to snake_case...")
    original_columns = df.columns.tolist()
    # Define mapping - **Adjust this based on your actual column names from the DB**
    new_column_names = {
        col: col.lower().replace(' ', '_').replace('(','').replace(')','').replace('.','') 
        for col in df.columns
    }
    # Example specific renaming if needed (uncomment and adjust if auto-rename isn't perfect)
    # new_column_names['Budget (Crores)'] = 'budget_crores' 
    # new_column_names['BoxOffice (Crores)'] = 'box_office_crores'
    # new_column_names['Duration (minutes)'] = 'duration_minutes'
    
    df.rename(columns=new_column_names, inplace=True)
    print("Columns renamed (using automatic conversion):")
    print(f"  Original: {original_columns}")
    print(f"  New:      {df.columns.tolist()}")
    
    # Store cleaned column names for subsequent steps
    # Adjust these variable names if your automatic renaming results in different keys
    genre_col = new_column_names.get('Genre', 'genre') # Get the new name for 'Genre'
    budget_col = new_column_names.get('Budget (Crores)', 'budget_crores') # Get the new name for 'Budget (Crores)'
    box_office_col = new_column_names.get('BoxOffice (Crores)', 'box_office_crores') # Get the new name




Step 2: Renaming columns to snake_case...
Columns renamed (using automatic conversion):
  Original: ['MovieID', 'Title', 'Director', 'Genre', 'ReleaseYear', 'Budget (Crores)', 'BoxOffice (Crores)', 'Rating', 'Duration (minutes)', 'LeadActor', 'LeadActress', 'Language', 'ProductionCompany']
  New:      ['movieid', 'title', 'director', 'genre', 'releaseyear', 'budget_crores', 'boxoffice_crores', 'rating', 'duration_minutes', 'leadactor', 'leadactress', 'language', 'productioncompany']


In [36]:
# Step 3: Handle missing 'Genre' (using the potentially renamed column)
if not df.empty and genre_col in df.columns:
    print(f"\nStep 3: Handling missing '{genre_col}' values...")
    missing_genre_before = df[genre_col].isna().sum()
    if missing_genre_before > 0:
        df[genre_col].fillna('Unknown', inplace=True)
        print(f"Filled {missing_genre_before} missing {genre_col} value(s) with 'Unknown'.")
    else:
        print(f"No missing {genre_col} values found.")
elif not df.empty:
     print(f"\nStep 3: Column '{genre_col}' not found, skipping genre imputation.")


Step 3: Handling missing 'genre' values...
Filled 1 missing genre value(s) with 'Unknown'.


/tmp/ipykernel_9902/3081186645.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[genre_col].fillna('Unknown', inplace=True)


In [37]:
# Step 4: Handle missing numerical columns using median (using potentially renamed columns)
if not df.empty:
    print("\nStep 4: Imputing missing numerical values with median...")
    
    # Impute Budget
    if budget_col in df.columns:
        # Convert column to numeric, coercing errors (like strings) to NaN
        df[budget_col] = pd.to_numeric(df[budget_col], errors='coerce')
        missing_budget_before = df[budget_col].isna().sum()
        if missing_budget_before > 0:
            budget_median = df[budget_col].median()
            df[budget_col].fillna(budget_median, inplace=True)
            print(f"Filled {missing_budget_before} missing {budget_col} value(s) with median ({budget_median:.2f}).")
        else:
            print(f"No missing {budget_col} values found (or all were non-numeric errors).")
    else:
        print(f"Column '{budget_col}' not found, skipping budget imputation.")

    # Impute Box Office
    if box_office_col in df.columns:
        # Convert column to numeric, coercing errors to NaN
        df[box_office_col] = pd.to_numeric(df[box_office_col], errors='coerce')
        missing_box_office_before = df[box_office_col].isna().sum()
        if missing_box_office_before > 0:
            box_office_median = df[box_office_col].median()
            df[box_office_col].fillna(box_office_median, inplace=True)
            print(f"Filled {missing_box_office_before} missing {box_office_col} value(s) with median ({box_office_median:.2f}).")
        else:
            print(f"No missing {box_office_col} values found (or all were non-numeric errors).")
    else:
         print(f"Column '{box_office_col}' not found, skipping box office imputation.")



Step 4: Imputing missing numerical values with median...
Filled 12 missing budget_crores value(s) with median (50.00).
Filled 9 missing boxoffice_crores value(s) with median (250.00).


/tmp/ipykernel_9902/2110447190.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[budget_col].fillna(budget_median, inplace=True)
/tmp/ipykernel_9902/2110447190.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [38]:
# Step 5: Strip whitespace from object (string) columns
if not df.empty:
    print("\nStep 5: Stripping leading/trailing whitespace from string columns...")
    string_columns = df.select_dtypes(include='object').columns
    stripped_cols_count = 0
    for col in string_columns:
        # Check if column still exists and is actually string type after potential coercions
        if col in df.columns and pd.api.types.is_string_dtype(df[col]):
            try:
                # Check if stripping changes anything to avoid unnecessary messages
                if df[col].str.strip().equals(df[col]):
                    continue # Skip if no change
                df[col] = df[col].str.strip()
                stripped_cols_count += 1
            except AttributeError:
                 # Handle cases where a column might contain non-string data mixed in
                 print(f"  Warning: Could not apply strip() to column '{col}'. May contain non-string data.")
                 
    if stripped_cols_count > 0:
         print(f"Whitespace stripped from {stripped_cols_count} string column(s).")
    else:
         print("No leading/trailing whitespace found or removed in string columns.")



Step 5: Stripping leading/trailing whitespace from string columns...
Whitespace stripped from 1 string column(s).


In [43]:
if not df.empty:
    print("\nStep 6: Checking for duplicate movie titles...")
    
    duplicate_count = df.duplicated(subset=['title']).sum()
    
    if duplicate_count > 0:
        print(f"Found {duplicate_count} duplicate movie title(s).")
        print("Dropping duplicates based on title...")
        df.drop_duplicates(subset=['title'], keep='first', inplace=True)
        print(f"{duplicate_count} duplicate title(s) dropped. New shape: {df.shape}")
    else:
        print("No duplicate movie titles found.")
        
    print("\n--- Data Cleaning Complete ---")
    print("-" * 30)


Step 6: Checking for duplicate movie titles...
Found 4 duplicate movie title(s).
Dropping duplicates based on title...
4 duplicate title(s) dropped. New shape: (51, 13)

--- Data Cleaning Complete ---
------------------------------


## 4. Inspect Cleaned Data State

Check the DataFrame after applying the cleaning steps.

In [44]:
# Only proceed if the DataFrame is not empty
if not df.empty:
    print("\n--- Cleaned Data ---")
    # Display the first 5 rows
    print("\nFirst 5 rows (Cleaned):")
    display(df.head())
    
    # Print cleaned info
    print("\nCleaned DataFrame Info:")
    df.info()
    
    # Print cleaned missing values (should be 0 for handled columns)
    print("\nCleaned Missing values per column:")
    print(df.isna().sum().sort_values(ascending=False))
    
    # Display descriptive statistics for numerical columns after cleaning
    print("\nDescriptive Statistics (Cleaned):")
    # Select only numeric columns for describe()
    display(df.describe(include=np.number))
else:
    print("DataFrame is empty. Cannot display cleaned state.")


--- Cleaned Data ---

First 5 rows (Cleaned):


,movieid,title,director,genre,releaseyear,budget_crores,boxoffice_crores,rating,duration_minutes,leadactor,leadactress,language,productioncompany
0,MOV001,Lagaan,Ashutosh Gowariker,Unknown,2001,50.0,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions
1,MOV002,Kabhi Khushi Kabhie Gham...,Karan Johar,Family Drama,2001,50.0,100.0,7.9,210,Shah Rukh Khan,Kajol,Hindi,Dharma Productions
2,MOV003,M.S. Dhoni: The Untold Story,Neeraj Pandey,Biographical Sports Drama,2016,50.0,215.0,7.9,184,Sushant Singh Rajput,Kiara Advani,Hindi,Fox Star Studios
3,MOV004,Baahubali 2: The Conclusion,S. S. Rajamouli,Epic Fantasy Action,2017,250.0,1810.0,8.2,171,Prabhas,Anushka Shetty,Telugu,Arka Media Works
4,MOV005,Chennai Express,Rohit Shetty,Action Comedy,2013,50.0,423.0,6.4,141,Shah Rukh Khan,Deepika Padukone,Hindi,Red Chillies Entertainment



Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 54
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movieid            51 non-null     object 
 1   title              51 non-null     object 
 2   director           51 non-null     object 
 3   genre              51 non-null     object 
 4   releaseyear        51 non-null     int64  
 5   budget_crores      51 non-null     float64
 6   boxoffice_crores   51 non-null     float64
 7   rating             51 non-null     float64
 8   duration_minutes   51 non-null     int64  
 9   leadactor          51 non-null     object 
 10  leadactress        51 non-null     object 
 11  language           51 non-null     object 
 12  productioncompany  51 non-null     object 
dtypes: float64(3), int64(2), object(8)
memory usage: 5.6+ KB

Cleaned Missing values per column:
movieid              0
title                0
director     

,releaseyear,budget_crores,boxoffice_crores,rating,duration_minutes
count,51.000000,51.000000,51.000000,51.000000,51.000000
mean,2015.784314,82.470588,338.784314,7.203922,156.372549
std,5.360275,98.224916,389.903440,0.892404,20.197981
min,1995.000000,1.000000,5.000000,4.400000,118.000000
25%,2015.000000,23.500000,100.000000,6.800000,144.500000
50%,2018.000000,50.000000,250.000000,7.400000,158.000000
75%,2019.000000,92.500000,409.000000,7.900000,166.500000
max,2020.000000,550.000000,2140.000000,8.400000,224.000000


In [45]:
df

,movieid,title,director,genre,releaseyear,budget_crores,boxoffice_crores,rating,duration_minutes,leadactor,leadactress,language,productioncompany
0,MOV001,Lagaan,Ashutosh Gowariker,Unknown,2001,50.0,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions
1,MOV002,Kabhi Khushi Kabhie Gham...,Karan Johar,Family Drama,2001,50.0,100.0,7.9,210,Shah Rukh Khan,Kajol,Hindi,Dharma Productions
2,MOV003,M.S. Dhoni: The Untold Story,Neeraj Pandey,Biographical Sports Drama,2016,50.0,215.0,7.9,184,Sushant Singh Rajput,Kiara Advani,Hindi,Fox Star Studios
3,MOV004,Baahubali 2: The Conclusion,S. S. Rajamouli,Epic Fantasy Action,2017,250.0,1810.0,8.2,171,Prabhas,Anushka Shetty,Telugu,Arka Media Works
4,MOV005,Chennai Express,Rohit Shetty,Action Comedy,2013,50.0,423.0,6.4,141,Shah Rukh Khan,Deepika Padukone,Hindi,Red Chillies Entertainment
5,MOV006,Bharat,Ali Abbas Zafar,Drama,2019,100.0,325.0,6.8,156,Salman Khan,Katrina Kaif,Hindi,Reel Life Productions
6,MOV007,Dangal,Nitesh Tiwari,Biographical Sports Drama,2016,70.0,2140.0,8.1,161,Aamir Khan,Fatima Sana Shaikh,Hindi,Aamir Khan Productions
7,MOV008,Sarkar (Tamil),A. R. Murugadoss,Action Thriller,2005,50.0,250.0,7.5,173,Vijay,Keerthy Suresh,Tamil,Kalaignar TV
8,MOV009,PK,Rajkumar Hirani,Satirical Science Fiction Comedy,2014,85.0,792.0,8.1,153,Aamir Khan,Anushka Sharma,Hindi,Vinod Chopra Films
9,MOV010,2.0 (Tamil),S. Shankar,Science Fiction Action,2018,550.0,800.0,6.9,147,Rajinikanth,Amy Jackson,Tamil,Lyca Productions


In [46]:
    df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 54
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movieid            51 non-null     object 
 1   title              51 non-null     object 
 2   director           51 non-null     object 
 3   genre              51 non-null     object 
 4   releaseyear        51 non-null     int64  
 5   budget_crores      51 non-null     float64
 6   boxoffice_crores   51 non-null     float64
 7   rating             51 non-null     float64
 8   duration_minutes   51 non-null     int64  
 9   leadactor          51 non-null     object 
 10  leadactress        51 non-null     object 
 11  language           51 non-null     object 
 12  productioncompany  51 non-null     object 
dtypes: float64(3), int64(2), object(8)
memory usage: 5.6+ KB


# Save Cleaned Data

In [47]:
df.to_csv('/cleaned_bollywood_movies.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'movies_data'